# Value gradient error for linear policies in LQG

Experiment description on [Overleaf](https://www.overleaf.com/read/cmbgmxxpxqzr).

**Versioning:** [CalVer](https://calver.org) `MM.DD.MICRO`

In [1]:
from __future__ import annotations

import logging
import os
import os.path as osp

import lqsvg
import lqsvg.envs.lqr.utils as lqg_util
import lqsvg.experiment.utils as utils
import lqsvg.torch.named as nt
import pytorch_lightning as pl
import ray
from lqsvg.experiment.data import build_datamodule
from lqsvg.experiment.models import LightningModel
from lqsvg.experiment.worker import make_worker
from ray import tune
from raylab.policy.model_based.lightning import LightningTrainerSpec
from torch import Tensor

import wandb

In [2]:
CWD = os.getcwd()

In [3]:
class InputStatistics(pl.callbacks.Callback):
    def on_train_batch_end(
        self,
        trainer: pl.Trainer,
        pl_module: pl.LightningModule,
        outputs: Tensor,
        batch: tuple[Tensor, Tensor, Tensor],
        batch_idx: int,
        dataloader_idx: int,
    ):
        del trainer, outputs, batch_idx, dataloader_idx
        obs, act, new_obs = batch
        pl_module.log("train/obs-mean", obs.mean())
        pl_module.log("train/obs-std", obs.std())
        pl_module.log("train/act-mean", act.mean())
        pl_module.log("train/act-std", act.std())
        pl_module.log("train/new_obs-mean", new_obs.mean())
        pl_module.log("train/new_obs-std", new_obs.std())

In [4]:
class Experiment(tune.Trainable):
    def setup(self, config: dict):
        self.run = wandb.init(
            dir=osp.join(CWD, ""),
            name="SVG Prediction",
            config=config,
            project="LQG-SVG",
            entity="angelovtt",
            tags=[utils.calver()],
            reinit=True,
            mode="online",
            save_code=True,
        )

        self.make_worker()
        self.make_model()
        self.make_datamodule()
        self.make_lightning_trainer()
        self.make_artifact()
        utils.suppress_lightning_info_logging()

    @property
    def hparams(self):
        return self.run.config

    def make_worker(self):
        with nt.suppress_named_tensor_warning():
            self.worker = make_worker(
                env_config=self.hparams.env_config,
                policy_config=self.hparams.policy,
                log_level=logging.WARNING,
            )

    def make_model(self):
        self.model = LightningModel(self.worker.get_policy(), self.worker.env)
        self.model.hparams.learning_rate = self.hparams.learning_rate
        self.model.hparams.mc_samples = self.hparams.mc_samples
        self.model.hparams.weight_decay = self.hparams.weight_decay

    def make_datamodule(self):
        self.datamodule = build_datamodule(
            self.worker, total_trajs=self.hparams.total_trajs
        )
        self.datamodule.collect_trajectories(prog=False)

    def make_lightning_trainer(self):
        logger = pl.loggers.WandbLogger(
            save_dir=self.run.dir, log_model=False, experiment=self.run
        )

        early_stopping = pl.callbacks.EarlyStopping(
            monitor=LightningModel.early_stop_on,
            min_delta=float(self.hparams.improvement_delta),
            patience=int(self.hparams.patience),
            mode="min",
            strict=True,
        )
        checkpointing = pl.callbacks.ModelCheckpoint(
            dirpath=osp.join(self.run.dir, "checkpoints"),
            monitor=LightningModel.early_stop_on,
            save_top_k=-1,
            period=10,
            save_last=True,
        )
        self.trainer = pl.Trainer(
            default_root_dir=self.run.dir,
            logger=logger,
            num_sanity_val_steps=2,
            callbacks=[early_stopping, checkpointing, InputStatistics()],
            max_epochs=self.hparams.max_epochs,
            progress_bar_refresh_rate=0,  # don't show progress bar for model training
            weights_summary=None,  # don't print summary before training
        )

    def make_artifact(self):
        env = self.worker.env
        self.artifact = wandb.Artifact(
            f"svg_prediction-lqg{env.n_state}.{env.n_ctrl}.{env.horizon}", type="model"
        )

    def step(self) -> dict:
        self.log_env_info()
        with utils.suppress_dataloader_warning():
            self.trainer.fit(self.model, datamodule=self.datamodule)

            results = self.trainer.test(self.model, datamodule=self.datamodule)[0]
            self.run.summary.update(results)

        self.artifact.add_dir(self.trainer.checkpoint_callback.dirpath)
        self.run.log_artifact(self.artifact)
        return {tune.result.DONE: True, **results}

    def log_env_info(self):
        dynamics = self.worker.env.dynamics
        eigvals = lqg_util.stationary_eigvals(dynamics)
        tests = {
            "stability": lqg_util.isstable(eigvals=eigvals),
            "controllability": lqg_util.iscontrollable(dynamics),
        }
        self.run.summary.update(tests)
        self.run.summary.update({"Fs_eigvals": wandb.Histogram(eigvals)})

    def cleanup(self):
        self.run.finish()

In [5]:
ray.init(logging_level=logging.WARNING)
lqsvg.register_all()
utils.suppress_lightning_info_logging()

config = {
    "env_config": dict(
        n_state=2,
        n_ctrl=2,
        horizon=20,
        stationary=True,
        Fs_eigval_range=(0.0, 1.0),
        transition_bias=False,
        num_envs=100,
    ),
    "policy": {
        "module": {
            "policy_initializer": "xavier_uniform",
            "model_initializer": "xavier_uniform",
            "stationary_model": True,
            "model_input_norm": tune.grid_search([True, False]),
        }
    },
    "learning_rate": 1e-3,
    "weight_decay": 1e-4,
    "mc_samples": 32,
    "total_trajs": 1000,
    "improvement_delta": 0.0,
    "patience": 3,
    "max_epochs": 1000,
}

analysis = tune.run(Experiment, config=config, num_samples=8, local_dir="./results")

Trial name,status,loc,policy/module/model_input_norm
Experiment_a089f_00000,RUNNING,,True


(pid=14001) wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)
(pid=14002) wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)
(pid=14004) wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)
(pid=14003) wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)
(pid=14004) wandb: ERROR Error while calling W&B API: Error 1213: Deadlock found when trying to get lock; try restarting transaction (<Response [500]>)
(pid=14001) wandb: Tracking run with wandb version 0.10.23
(pid=14001) wandb: Syncing run SVG Prediction
(pid=14001) wandb: ⭐️ View project at https://wandb.ai/angelovtt/LQG-SVG
(pid=14001) wandb: 🚀 View run at https://wandb.ai/angelovtt/LQG-SVG/runs/9vib5hut
(pid=14001) wandb: Run data is saved locally in /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/run-20210325_163106-9vib5hut
(pid=14001) wandb: Run `wandb offline` to 

(pid=14001) 
(pid=14003) 
(pid=14002) 


(pid=14001) 2021-03-25 16:31:09,501	WARNING deprecation.py:33 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=14002) 2021-03-25 16:31:09,491	WARNING deprecation.py:33 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=14003) 2021-03-25 16:31:09,489	WARNING deprecation.py:33 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=14004) wandb: Tracking run with wandb version 0.10.23
(pid=14004) wandb: Syncing run SVG Prediction
(pid=14004) wandb: ⭐️ View project at https://wandb.ai/angelovtt/LQG-SVG
(pid=14004) wandb: 🚀 View run at https://wandb.ai/angelovtt/LQG-SVG/runs/3dfzmi0p
(pid=14004) wandb: Run data is saved locally in /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/run-20210325_163106-3dfzmi0p
(pid=14004) wandb: Run `wandb 

(pid=14004) 


...

(pid=14719) --------------------------------------------------------------------------------
(pid=14719) DATALOADER:0 TEST RESULTS
(pid=14719) {'test/analytic_cossim': tensor(-0.0902),
(pid=14719)  'test/analytic_diff': tensor(4.0318),
(pid=14719)  'test/analytic_svg_norm': tensor(287.7411),
(pid=14719)  'test/analytic_value': tensor(-245.2268),
(pid=14719)  'test/loss': tensor(37.9452),
(pid=14719)  'test/monte_carlo_cossim': tensor(-0.1077),
(pid=14719)  'test/monte_carlo_diff': tensor(-3.4216),
(pid=14719)  'test/monte_carlo_svg_norm': tensor(312.3213),
(pid=14719)  'test/monte_carlo_value': tensor(-252.6802),
(pid=14719)  'true_svg_norm': tensor(102.8296),
(pid=14719)  'true_value': tensor(-249.2586)}
(pid=14719) --------------------------------------------------------------------------------
Result for Experiment_a089f_00015:
  date: 2021-03-25_16-43-11
  done: true
  experiment_id: 3ab43f871df74fd3a76ea5b1dcac1d73
  hostname: Angelos-MBP
  iterations_since_restore: 1
  node_ip: 1

Trial name,status,loc,policy/module/model_input_norm,iter,total time (s),test/loss,true_value,true_svg_norm
Experiment_a089f_00000,TERMINATED,,True,1,105.798,,-594.717,1737.42
Experiment_a089f_00001,TERMINATED,,False,1,157.194,,-65.6031,89.1708
Experiment_a089f_00002,TERMINATED,,True,1,96.4929,,-134.075,124.17
Experiment_a089f_00003,TERMINATED,,False,1,118.327,,-90.8687,74.5132
Experiment_a089f_00004,TERMINATED,,True,1,63.3313,,-64.4971,30.5406
Experiment_a089f_00005,TERMINATED,,False,1,147.256,,-70.4535,86.387
Experiment_a089f_00006,TERMINATED,,True,1,373.204,,-1854.31,12634.5
Experiment_a089f_00007,TERMINATED,,False,1,232.198,,-1389.03,8908.1
Experiment_a089f_00008,TERMINATED,,True,1,168.498,,-378.231,309.285
Experiment_a089f_00009,TERMINATED,,False,1,108.634,,-150.102,260.374


(pid=14719) wandb: Waiting for W&B process to finish, PID 14730
(pid=14719) wandb: Program ended successfully.
(pid=14639) wandb: - 8.94MB of 8.94MB uploaded (0.00MB deduped)
(pid=14719) wandb: - 8.68MB of 8.68MB uploaded (0.00MB deduped)
(pid=14639) wandb: \ 10.09MB of 10.12MB uploaded (0.00MB deduped)
(pid=14719) wandb: \ 9.51MB of 9.51MB uploaded (0.00MB deduped)
wandb:                                                                                
(pid=14639) wandb: Find user logs for this run at: /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/run-20210325_164014-2z7f9pqn/logs/debug.log
(pid=14639) wandb: Find internal logs for this run at: /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/run-20210325_164014-2z7f9pqn/logs/debug-internal.log
(pid=14639) wandb: Run summary:
(pid=14639) wandb:                   stability True
(pid=14639) wandb:             controllability True
(pid=14639) wandb:                    val/loss 63.27656
(pid=14639)

(pid=14639) 


(pid=14719) wandb: Find user logs for this run at: /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/run-20210325_164115-2in0bd3m/logs/debug.log
(pid=14719) wandb: Find internal logs for this run at: /Users/angelolovatto/Repositories/personal/LQG-SVG/experiments/wandb/run-20210325_164115-2in0bd3m/logs/debug-internal.log
(pid=14719) wandb: Run summary:
(pid=14719) wandb:                   stability True
(pid=14719) wandb:             controllability True
(pid=14719) wandb:                    val/loss 38.32711
(pid=14719) wandb:       val/monte_carlo_value -277.2457
(pid=14719) wandb:    val/monte_carlo_svg_norm 332.45038
(pid=14719) wandb:        val/monte_carlo_diff -27.98708
(pid=14719) wandb:      val/monte_carlo_cossim -0.11457
(pid=14719) wandb:          val/analytic_value -245.22684
(pid=14719) wandb:       val/analytic_svg_norm 287.74115
(pid=14719) wandb:           val/analytic_diff 4.03178
(pid=14719) wandb:         val/analytic_cossim -0.09025
(pid=14719) wa

In [6]:
ray.shutdown()